# Benchmark Data Sets in Exam Timetabling
Data -> __[University of Toronto Benchmark Data](http://www.cs.nott.ac.uk/~pszrq/data.htm)__

**Problem Description :**
... A Conflict Matrix C was defined where each element Cij = 1 if exam i conflict with exam j (have common students), or Cij = 0 otherwise. The Conflict Density represents the ratio between the number of elements of value "1" to the total number of elements in the conflict matrix.

**Objective :**
1. to minimise the number of timeslots needed for the problem (graph coloring)
2. to minimise the sum of approximate costs per student. i.e. Zigma wi/S, i = 0, 1, 2, 3, 4.

**LIST TO DO**
- [x] Import Data
- [x] Build Conflict Matrix
- [x] Make TIme Table


In [1]:
# Import Library
import pandas as pd
import numpy as np
import random
import copy
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# pd.reset_option('^display.', silent=True)

In [2]:
# Import Data
import timeit
start_time = timeit.default_timer()
course_path = "D:/Dataset/okh1/car-f-92.crs"
student_path = "D:/Dataset/okh1/car-f-92.stu"

course = pd.read_csv(course_path, header = None, sep = " ")
student = pd.read_csv(student_path, header = None, sep = " ", 
                      names = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15'], skip_blank_lines = False)
student = student.fillna(0)

In [3]:
for i in student:
    if sum(student[i]) == 0:
        student = student.drop(columns = i)
#student

In [4]:
# Build Matrix in DataFrame
start_time = timeit.default_timer()
matrix = pd.DataFrame(0, index = range(1, (len(course)+1)), columns = range(1, (len(course)+1)), dtype = np.int)
for stud in range(0, len(student)):
    for i in range(0, len(student.columns)-1):
        for j in range(i+1, len(student.columns)):
            a = int(student.iloc[stud, i])
            b = int(student.iloc[stud, j])
            if b == 0 or a == b:
                break
            matrix.iloc[a-1, b-1] += 1
            matrix.iloc[b-1, a-1] += 1
        if student.iloc[stud, i] == 0:
            break
            
# Run to open matrix
# matrix
elapsed = timeit.default_timer() - start_time
elapsed

48.3698453

**Build Time Tabling Using The Largest Degree First Heuristic**

In [5]:
degree = 0
table_of_degree = []
course_chaos = []
for i in range(0, len(course)):
    for j in matrix:
        if matrix.iloc[i][j] > 0:
            degree += 1
            chaos = j
            course_chaos.append(chaos)
    key = (i+1, degree, course_chaos)
    table_of_degree.append(key)
    degree = 0
    course_chaos = []
degree_table = pd.DataFrame(table_of_degree, columns = ['course', 'total_degree', 'course_chaos'])
degree_table = degree_table.set_index('course')
#degree_table

In [6]:
sorted_degree_table = degree_table.sort_values(by = ['total_degree'], ascending = False)
# sorted_degree_table

In [7]:
time_table = [[]]
for i in sorted_degree_table.index[:len(course)+1]:
    arr = degree_table.iloc[i-1, 1]
    for j in range(0, len(time_table)):        
        if any(a in arr for a in time_table[j]):
            if j == (len(time_table)-1):
                time_table.append([i])
            continue
        else:
            time_table[j].append(i)
            break
#time_table

In [8]:
# code you want to evaluate
elapsed = timeit.default_timer() - start_time
elapsed

94.81394110000001

In [9]:
# slot = []
# for i in range(1, len(time_table)+1):
#     slot.append('Slot {list_slot}'.format(list_slot = i))

    
# pd.set_option('display.max_rows', None)
# solution = pd.Series(time_table, index = slot)
# display(solution)

In [10]:
# for i in range(0, len(solution)):
#     print('slot ', i+1)
#     print(solution[i])
#     print('\n')

In [11]:
arr_time_table = []
def index_2d(myList, v):
    for i, x in enumerate(myList):
        if v in x:
            return i

for i in range(1, len(degree_table)+1):
    index = index_2d(time_table , i)
    arr_time_table.append([i, index])

In [12]:
# for i in range(0, len(time_table)):
#     for j in range(0, len(time_table[i])):
#         print(time_table[i][j], i)

In [13]:
student_array = [[]]
for i in student.index:
    for j in range(0, len(student.iloc[i])):
        if student.iloc[i][j] > 0:
            if len(student_array) <= i:
                student_array.append([student.iloc[i][j]])
            else:
                student_array[i].append(student.iloc[i][j])

In [14]:
def check_hard_constraint(initial_solution, course, index):
    check = True
    arr = degree_table.iloc[course-1, 1]
    exam = []
    for i in range(0, len(initial_solution)):
        if arr_time_table[i][1] == index:
            exam.append(i+1)
    if any(a in arr for a in exam):
        check = False
    return bool(check)

def evaluate(initial_solution):
    exam_score = 0
    for i in range(1, len(degree_table)+1):
        index_a = initial_solution[i-1][1]
        for j in range(i+1, len(degree_table)):
            index_b = initial_solution[j-1][1]
            diff = abs(index_a - index_b)
            num = matrix.iloc[i-1, j-1]
            if diff == 1:
                exam_score += 16*num
            elif diff == 2:
                exam_score += 8*num
            elif diff == 3:
                exam_score += 4*num
            elif diff == 4:
                exam_score += 2*num
            elif diff == 5:
                exam_score += 1*num
        
    return exam_score/len(student_array) 

def get_cost(initial_solution, course):
    cost = 0
    index_a = initial_solution[course-1][1]
    for i in range(1, len(degree_table)+1):
        if i == course:
            continue
        index_b = initial_solution[i-1][1]
        diff = abs(index_a - index_b)
        num = matrix.iloc[i-1, course-1]
        if diff == 1:
            cost += 16*num
        elif diff == 2:
            cost += 8*num
        elif diff == 3:
            cost += 4*num
        elif diff == 4:
            cost += 2*num
        elif diff == 5:
            cost += 1*num
    return cost/len(student_array) 

Carleton91 = 11.694830132939439
Carleton92 = 10.407080
EarlHaig83 = 71.91466666666666
EdHEC92 = 27.799858306765852
KingFahd93 = 46.339502710787066
LSE91 = 26.202494497432134
Rye92 = 34.06339806670731
Andrews83 = 193.7315875613748
TorontoAS92 = 7.444418320323521
TorontoE92 = 53.447272727272725
Trent92 = 15.687385321100917
YorkMills83 = 64.18597236981934
Pur93 = 16.680673947789025

slot_Carleton91 = 35-1
slot_Carleton92 = 32-1
slot_EarlHaig83 = 24-1
slot_EdHEC92 = 18-1
slot_KingFahd93 = 20-1
slot_LSE91 = 18-1
slot_Rye92 = 23-1
slot_Andrews83 = 13-1
slot_TorontoAS92 = 35-1
slot_TorontoE92 = 10-1
slot_Trent92 = 23-1
slot_YorkMills83 = 21-1
slot_Pur93 = 42-1

## Algoritma Hill Climbing
***

In [15]:
# start_time = timeit.default_timer()
# print(evaluate(arr_time_table))
# elapsed = timeit.default_timer() - start_time
# elapsed 

In [16]:
best_score = Carleton92
time_slot = slot_Carleton92

In [ ]:
start_time = timeit.default_timer()

block = 0
while block < 1000:
    block += 1
    print('Iterasi', block, 'dengan penalty', best_score)
    course = int(random.randint(1, len(degree_table.index)))
    index = int(random.randint(0, time_slot))
    penalty_1 = get_cost(arr_time_table, course)
    if check_hard_constraint(arr_time_table, course, index):
        new_solution = copy.deepcopy(arr_time_table)
        new_solution[course-1][1] = index 
        penalty_2 = get_cost(new_solution, course)
    else:   
        penalty_2 = penalty_1
        
    delta = penalty_1 - penalty_2
    
    if delta > 0:
        score = best_score - penalty_2
        best_score = score
        arr_time_table = copy.deepcopy(new_solution)
    else:
        score = 1000

        
# arr_time_table
elapsed = timeit.default_timer() - start_time
elapsed 

Iterasi 1 dengan penalty 10.40708
Iterasi 2 dengan penalty 10.40708
Iterasi 3 dengan penalty 10.40708
Iterasi 4 dengan penalty 10.40708
Iterasi 5 dengan penalty 10.40708
Iterasi 6 dengan penalty 10.40708
Iterasi 7 dengan penalty 10.40708
Iterasi 8 dengan penalty 10.40708
Iterasi 9 dengan penalty 10.40708
Iterasi 10 dengan penalty 10.40708
Iterasi 11 dengan penalty 10.40708
Iterasi 12 dengan penalty 10.32971388240404
Iterasi 13 dengan penalty 10.32971388240404
Iterasi 14 dengan penalty 10.32971388240404
Iterasi 15 dengan penalty 10.32971388240404
Iterasi 16 dengan penalty 10.32971388240404
Iterasi 17 dengan penalty 10.32971388240404
Iterasi 18 dengan penalty 10.32971388240404
Iterasi 19 dengan penalty 10.32971388240404
Iterasi 20 dengan penalty 10.32971388240404
Iterasi 21 dengan penalty 10.32971388240404
Iterasi 22 dengan penalty 10.32971388240404
Iterasi 23 dengan penalty 10.32971388240404
Iterasi 24 dengan penalty 10.320049948422824
Iterasi 25 dengan penalty 10.320049948422824
Iteras

In [ ]:
elapsed

In [ ]:
print(evaluate(arr_time_table))

In [ ]:
for i in range(0, len(arr_time_table)):
    print(arr_time_table[i][0], arr_time_table[i][1])

In [ ]:
percentage_delta = ((Carleton92-evaluate(arr_time_table))/Carleton92)*100
percentage_delta